In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skrebate import ReliefF
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import os
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from skrebate import ReliefF
from sklearn.metrics import make_scorer, accuracy_score

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from numpy import mean

def apply_resampling_pipeline(filepath):
    print(f"Traitement du fichier {filepath}...")
    try:
        data = pd.read_csv(filepath)
        data_filtered = data.dropna(axis=1)
        data_filtered = data_filtered[data_filtered['label'] != 'transitionNoFog']
        X = data_filtered.drop('label', axis=1)
        y = data_filtered['label'].apply(lambda x: 1 if x in ['fog', 'transitionFog'] else 0)

        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, shuffle=True, stratify=y, random_state=42)

        # Calculate class counts
        class_counts = y_train.value_counts()
        # Check class balance in the training set
        if class_counts.get(1, 0) >= class_counts.get(0, 0):
            print(f"No resampling needed for {filepath}, because class 1 count >= class 0 count")
            
            # Evaluate pipeline
            cv = StratifiedKFold(n_splits=10, shuffle=True, stratify = y_train, random_state=42)
            scores = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
            mean_score = mean(scores)
            
            return {'File': filepath, 'SMOTE Strategy': 'None', 'ROC AUC Score': mean_score, 'Note': 'No resampling due to class 1 >= class 0'}
        else:
            # Setup resampling and model
            # smote = SMOTE(sampling_strategy=smote_strategy, random_state=42)
            over_sampler = BorderlineSMOTE(sampling_strategy=1, random_state=42)
            model = DecisionTreeClassifier(random_state=42)
            steps = [('over_sampler', over_sampler), ('model', model)]
            pipeline = Pipeline(steps)

            # Evaluate pipeline
            cv = StratifiedKFold(n_splits=10, shuffle=True, stratify = y_train, random_state=42)
            scores = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
            mean_score = mean(scores)
            return {'File': filepath, 'Over Strategy': 1, 'ROC AUC Score': mean_score, 'Note': 'Resampling applied'}

    except Exception as e:
        print(f"Error processing {filepath}: {str(e)}")
        return None

# Path to directory
directory_path = 'C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/'

over_strategies = 1

# DataFrame to store results
results_df = pd.DataFrame()

# Process each file with each combination of resampling strategies
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory_path, filename)
        result = apply_resampling_pipeline(filepath)
        results_df = results_df.append(result, ignore_index=True)
        print(f"Added results for {filename} with OVER {1} to the DataFrame.")

# Filter out the rows where ROC AUC Score is 'N/A'
valid_results = results_df[results_df['ROC AUC Score'] != 'N/A']

# Find the best combination based on ROC AUC Score
if not valid_results.empty:
    best_combination = valid_results.loc[valid_results['ROC AUC Score'].idxmax()]
    print(f"The best resampling combination is UNDER {best_combination['Under Strategy']} with an ROC AUC Score of {best_combination['ROC AUC Score']}")

# Save results
results_file = 'C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/resultats_resampling/results_combinations_over_100.csv'
results_df.to_csv(results_file, index=False)
print(f"Results saved to {results_file}.")


In [3]:
def apply_vc_resampling_pipeline(filepath, smote_strategy, under_strategy):
    print(f"Démarrage du traitement pour {filepath} avec SMOTE {smote_strategy} et UNDER {under_strategy}")
    try:
        data = pd.read_csv(filepath)
        data_filtered = data.dropna(axis=1)
        data_filtered = data_filtered[data_filtered['label'] != 'transitionNoFog']
        X = data_filtered.drop('label', axis=1)
        y = data_filtered['label'].apply(lambda x: 1 if x in ['fog', 'transitionFog'] else 0)
        X_scaled = StandardScaler().fit_transform(X)
        X_train, _, y_train, _ = train_test_split(X_scaled, y, test_size=0.3, shuffle=True, stratify=y, random_state=42)
        
        model = DecisionTreeClassifier(random_state=42)
        steps = [('model', model)]
        if y_train.value_counts().get(1, 0) < y_train.value_counts().get(0, 0):
            steps = [('smote', SMOTE(sampling_strategy=smote_strategy, random_state=42)),
                     ('under_sampler', RandomUnderSampler(sampling_strategy=under_strategy, random_state=42))] + steps
        pipeline = Pipeline(steps)
        scores = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42), n_jobs=-1)
        print(f"Traitement terminé pour {filepath} avec un score ROC AUC moyen de {mean(scores)}")
        return {
            'File': filepath,
            'SMOTE Strategy': smote_strategy if 'smote' in steps[0] else 'None',
            'Under Strategy': under_strategy if 'under_sampler' in steps[1] else 'None',
            'ROC AUC Score': mean(scores),
            'Note': 'No resampling due to class 1 >= class 0' if 'model' in steps[0] else 'Resampling applied'
        }
    except Exception as e:
        error_message = "Ratio impossible" if "The specified ratio" in str(e) else str(e)
        print(f"Erreur lors du traitement de {filepath}: {error_message}")
        return {'File': filepath, 'Error': error_message}
    


In [7]:
# Path to directory
directory_path = 'C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/'

# Stratégies de resampling à tester
smote_strategies = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,1]
under_strategies = [0.1,0.2,0.3,0.4, 0.5, 0.6, 0.7,0.8,0.9,1]

# DataFrame to store results
results_df = pd.DataFrame()

# Process each file with each combination of resampling strategies
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory_path, filename)
        for smote_strategy in smote_strategies:
            for under_strategy in under_strategies:
                result = apply_vc_resampling_pipeline(filepath, smote_strategy, under_strategy)
                results_df = results_df.append(result, ignore_index=True)
                print(f"Added results for {filename} with SMOTE {smote_strategy} and UNDER {under_strategy} to the DataFrame.")

# Find the best combination based on ROC AUC Score
best_combination = results_df.loc[results_df['ROC AUC Score'].idxmax()]
print(f"The best resampling combination is SMOTE {best_combination['SMOTE Strategy']} and UNDER {best_combination['Under Strategy']} with an ROC AUC Score of {best_combination['ROC AUC Score']}")

# Save results
results_file = 'C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/resultats_resampling/results_combinations_TousLesFichiers.csv'
results_df.to_csv(results_file, index=False)
print(f"Results saved to {results_file}.")

Démarrage du traitement pour C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/A_P_1956-02-21_OFF_OFF_all_extraction_features.csv avec SMOTE 0.1 et UNDER 0.1
Erreur lors du traitement de C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/A_P_1956-02-21_OFF_OFF_all_extraction_features.csv: name 'cross_val_score' is not defined
Added results for A_P_1956-02-21_OFF_OFF_all_extraction_features.csv with SMOTE 0.1 and UNDER 0.1 to the DataFrame.
Démarrage du traitement pour C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/A_P_1956-02-21_OFF_OFF_all_extraction_features.csv avec SMOTE 0.1 et UNDER 0.2


C:\Users\antho\AppData\Local\Temp\ipykernel_14612\1482242868.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


Erreur lors du traitement de C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/A_P_1956-02-21_OFF_OFF_all_extraction_features.csv: name 'cross_val_score' is not defined
Added results for A_P_1956-02-21_OFF_OFF_all_extraction_features.csv with SMOTE 0.1 and UNDER 0.2 to the DataFrame.
Démarrage du traitement pour C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/A_P_1956-02-21_OFF_OFF_all_extraction_features.csv avec SMOTE 0.1 et UNDER 0.3


C:\Users\antho\AppData\Local\Temp\ipykernel_14612\1482242868.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


Erreur lors du traitement de C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/A_P_1956-02-21_OFF_OFF_all_extraction_features.csv: name 'cross_val_score' is not defined
Added results for A_P_1956-02-21_OFF_OFF_all_extraction_features.csv with SMOTE 0.1 and UNDER 0.3 to the DataFrame.
Démarrage du traitement pour C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/A_P_1956-02-21_OFF_OFF_all_extraction_features.csv avec SMOTE 0.1 et UNDER 0.4


C:\Users\antho\AppData\Local\Temp\ipykernel_14612\1482242868.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


Erreur lors du traitement de C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/A_P_1956-02-21_OFF_OFF_all_extraction_features.csv: name 'cross_val_score' is not defined
Added results for A_P_1956-02-21_OFF_OFF_all_extraction_features.csv with SMOTE 0.1 and UNDER 0.4 to the DataFrame.
Démarrage du traitement pour C:/Users/antho/Documents/MEMOIRE_M2/c3d_audeline/ON_OFF_all_features/A_P_1956-02-21_OFF_OFF_all_extraction_features.csv avec SMOTE 0.1 et UNDER 0.5


C:\Users\antho\AppData\Local\Temp\ipykernel_14612\1482242868.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


KeyboardInterrupt: 